# 对文档建立索引并实现检索及排序 

@Title: 对文档建立索引并实现检索及排序   
@Author: JJYDXFS   
@Date: 11 July 2021

In [106]:
import jieba
import os
import numpy as np
import time

In [107]:
def read_file(file_path):
    '''
    读文件内容
    '''
    fp = open(file_path,"r",encoding='gbk',errors='ignore')
    content = fp.read()
    fp.close()
    return content

def create_stopwords(file_path):
    '''
    创建停词列表
    '''
    stopwords = [line.strip() for line in open(file_path, 'r', encoding='utf-8').readlines()]
    return stopwords

def preprocess(content):
    '''
    对文本文件预处理
    '''
    content = content.replace("\n", "")
    content = content.replace("\u3000", "")
    
    # content = content.replace(" ", "")
    return content

def create_seq_index(start_doc_id, end_doc_id):
    '''
    建立顺序索引
    '''
    seq_index={}
    for doc_id in range(start_doc_id, end_doc_id, 1):
        raw_content = read_file(corpus_path+str(doc_id)+".txt").strip()
        content = preprocess(raw_content)
        content_seg = jieba.cut(content)    # jieba分词
        
        word_map = {} # 定义单文档词项表
        
        # 去停词 + 计算出现次数
        word_amount = 0
        for word in content_seg:
            word_amount+=1
            if word not in stopwords:
                if word not in word_map.keys():
                    word_map[word]=1
                else:
                    word_map[word]+=1
        # 计算 tf
        for word in word_map:
            # word_map[word]/=word_amount
            word_map[word] = round(word_map[word]/word_amount,8)
        # 存入顺排索引
        seq_index[doc_id]=word_map
        
    return seq_index

def Invert_in_batch(seq_index):
    '''
    对每块文档建立倒排索引
    '''
    
    global word_id_map
    global word_id_counting
    global word_id_table
    
    # 由顺序索引建立倒排索引
    tmp_word_table = {} # 定义 当前文档块 的词项表
    pos_table = {} # 定义 当前文档块 的倒排记录表
    for doc_id in seq_index: # 遍历顺序索引建立倒排索引
        for word in seq_index[doc_id]:
            if word not in word_id_map: # 全局词项表未收录词
                word_id_map[word] =  word_id_counting # 加入全局词项表映射
                word_id_table[word_id_map[word]] = 1 # 加入全局词项表
                word_id_counting += 1 # 序号自增
            
            word_id = word_id_map[word] # 获取词项表对应序号
            
            if word_id not in tmp_word_table: # 局部词项表未收录词
                tmp_word_table[word_id] = True # 加入局部词项表
                pos_table[word_id] = {doc_id:seq_index[doc_id][word]} # 创建倒排记录
            else:
                word_id_table[word_id] +=1
                pos_table[word_id][doc_id] = seq_index[doc_id][word] # 更新倒排记录
    
    return pos_table

## 1. 预处理工作

In [108]:
# 创建停词列表
stopwords_file = "F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\cn_stopwords.txt"
stopwords=create_stopwords(stopwords_file)

## 2. 建立倒排索引

### 2.1 基于内存建立倒排索引

In [109]:
time_start=time.time()

if __name__=="__main__":
    # corpus_path = 'F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\testdata\\' # 文本语料路径
    corpus_path = 'F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\data\\Sogou\\' # 文本语料路径
    start_doc_id, end_doc_id = 1, 2001 # 指定文档集范围，左闭右开
    doc_number = end_doc_id - start_doc_id # 文档总数
    
    # 建立顺序索引
    seq_index=create_seq_index(start_doc_id, end_doc_id)
    
    # 由顺序索引建立倒排索引
    word_table = {} # 定义词项表
    invert_table = {} # 定义倒排记录表
    for doc_id in seq_index: # 遍历顺序索引建立倒排索引
        for word in seq_index[doc_id]:
            if word not in word_table: # 未收录词
                word_table[word] = 1 # 加入词项表
                invert_table[word] = {doc_id:seq_index[doc_id][word]} # 创建倒排记录
            else:
                word_table[word] +=1
                invert_table[word][doc_id] = seq_index[doc_id][word]
    
    # 计算 idf
    for word in word_table:
        word_table[word] = round(word_table[word]/doc_number, 8)
                
time_end=time.time()
print('time cost',time_end-time_start,'s')

time cost 23.06274104118347 s


### 2.2 基于外部磁盘建立倒排索引（分块处理）

In [15]:
time_start=time.time()

if __name__=="__main__":
    # corpus_path = 'F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\testdata\\' # 文本语料路径
    corpus_path = 'F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\data\\Sogou\\' # 文本语料路径
    output_path = 'F:\\OneDrive\\Documents\\ThirdYear\\MediaDataAnalysis\\SearchEngine\\output\\' # 输出索引文件路径
    start_doc_id, end_doc_id = 10, 2000 # 指定文档集范围，左闭右开
    doc_number = end_doc_id - start_doc_id # 文档总数
    
    batch = 10 # 指定块数
    
    assert (end_doc_id - start_doc_id) % batch == 0 # 保证文档总数是batch的整数倍
    batch_size = int((end_doc_id - start_doc_id)/batch) # 分块进行处理
    
    # 全局变量
    word_id_map ={} # 维护 词项及其序号映射
    word_id_table = {} # 定义词项表
    word_id_counting = 0 # 词项自增序号
    
    tmp_doc_id = start_doc_id # 指定每块起始文档
    batch_count = 0 # 块计数器
    
    while batch_count != batch: # 还有块未处理完时则继续
        # 建立顺序索引
        seq_index = create_seq_index(tmp_doc_id, tmp_doc_id + batch_size)
        # 建立倒排索引
        tmp_pos_table = Invert_in_batch(seq_index)
        # 将块的倒排索引写入磁盘
        np.save(output_path+str(batch_count) + ".npy", tmp_pos_table)
        batch_count += 1
        # 读下一块
        tmp_doc_id += batch_size
        
    # 合并倒排索引
    invert_table = np.load(output_path+"0.npy",allow_pickle=True)[()] # 将nd array转为内置字典
    for table in range(1,batch):
        invert_table_tmp = np.load(output_path + str(table) + ".npy",allow_pickle=True)[()]
        for word_id in invert_table_tmp:
            if word_id in invert_table.keys(): # 若待合并项在原索引表中则更新原表项
                invert_table[word_id].update(invert_table_tmp[word_id])
            else: # 若待合并项不在原索引表中则新增一项
                invert_table[word_id] = invert_table_tmp[word_id]
    
    # 输出合并后的完整倒排索引
    # np.save(output_path+ "result.npy", invert_table)
    
    # 计算 idf
    for word_id in word_id_table:
        word_id_table[word] = round(word_id_table[word_id]/doc_number, 8)
                
time_end=time.time()
print('time cost',time_end-time_start,'s')

time cost 225.83593440055847 s


In [56]:
#print(invert_table_dict[word_id_map['教育']])
#print(sorted(invert_table[word_id_map['孩子']].items(), key = lambda kv:(kv[1], kv[0])))
#print(invert_table[word_id_map['教育']])
#np.save(output_path+ "result.npy", invert_table)

## 3. 布尔检索

In [76]:
def get_record(word):
    '''
    获得词项的倒排记录集合
    异常处理待完善
    '''
    try:
        result = set(list(invert_table[word].keys()))
    except Exception as e:
        raise e
        
    return result

def get_precede(op1, op2):
    '''
    返回两运算符优先级关系
    @param
    op1: 栈顶元素
    op2: 栈外元素
    @return
    优先级关系: > < = !
    '''
    
    if op2 == '#': return '>'
    
    precede_map={
        '(':{'(':'<', ')':'=', 'AND':'<', 'OR':'<', 'ANDNOT':'<'},
        ')':{'(':'!', ')':'>', 'AND':'>', 'OR':'>', 'ANDNOT':'>'},
        'AND':{'(':'<', ')':'>', 'AND':'>', 'OR':'>', 'ANDNOT':'>'},
        'OR':{'(':'<', ')':'>', 'AND':'<', 'OR':'>', 'ANDNOT':'<'},
        'ANDNOT':{'(':'<', ')':'>', 'AND':'>', 'OR':'>', 'ANDNOT':'>'}
    }

    return precede_map[op1][op2]

def is_op(op):
    '''
    判断是否为运算符
    '''
    return (op == '(' or op == ')' or op == 'AND' or op == 'OR' or op == 'ANDNOT')

def calc_record(record1, record2, op):
    '''
    按运算符对倒排记录执行计算
    '''
    if op == 'AND':
        return record1 & record2
    elif op == 'OR':
        return record1 | record2
    elif op == 'ANDNOT':
        return record2 - record1
    else:
        # 非法输入
        return

In [51]:
def calc_bool_exp(exp):
    '''
    计算布尔表达式
    @param 
    exp: 布尔表达式的列表形式
    @return
    result[-1]: 总体交集
    word_set: 涉及的词项集合
    '''
    result_stack=[] # 预算结果栈
    op_stack=[] # 运算符栈
    word_set = set([]) # 词项集合
    i = 0 # 计数器i
    elen = len(exp) # 表达式长度

    while i < elen or len(op_stack) != 0:

        if i < elen and not is_op(exp[i]): # 词项入栈
            result_stack.append(get_record(exp[i]))
            word_set.add(exp[i]) # 词项入集合
            i += 1
        elif i<elen and len(op_stack) == 0: # 第一个运算符入栈
            op_stack.append(exp[i])
            i += 1
        else:
            op1 = op_stack[-1] # 取运算符栈顶元素
            # 取当前运算符，若表达式结束，则返回 '#'
            c = exp[i] if i < elen else '#'
            # 判断栈顶和当前运算符的优先级
            precede = get_precede(op1, c)
            if precede == '<':
                op_stack.append(c)
                i += 1
            elif precede == '=':
                op_stack.pop()
                i += 1
            elif precede == '>':
                op_stack.pop() # 栈顶运算符出栈
                record1 = result_stack.pop() # 中间结果1出栈
                record2 = result_stack.pop() # 中间结果2出栈
                result = calc_record(record1, record2, op1) 
                result_stack.append(result) # 计算结果入栈
            else:
                # 优先级错误
                return

    return result_stack[-1], word_set

In [34]:
# 测试对照程序
edu = set(list(invert_table['教育'].keys()))
stu = set(list(invert_table['学生'].keys()))
child = set(list(invert_table['孩子'].keys()))

In [69]:
len((stu - child) | (edu - child))

40

In [140]:
# 测试驱动：输入表达式字符串
# exp_str = '孩子'
exp_str = input('请输入检索表达式：')
exp=exp_str.split(' ')
# 运行计算程序
search_result, word_set = calc_bool_exp(exp)

请输入检索表达式：大学 AND 高中


In [141]:
print(search_result)

{741}


In [125]:
print(word_set)

{'中学', '大学'}


## 4. 基于TF/IDF排序对检索结果进行排序

In [143]:
sorted_result = {}

for doc in search_result: # 计算每个文档的 tfidf 值
    tfidf = 0
    for word in word_set:
        # 词没有对应文档倒排记录的，tf置为 0 
        tf = 0 if doc not in invert_table[word].keys() else invert_table[word][doc]
        idf = word_table[word]
        tfidf += round(tf*idf, 10)
    sorted_result[doc] = tfidf

# 降序排列
sorted(sorted_result.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

[(741, 4.7917e-06)]